In [11]:
import pipeline

In [12]:
filepath = '../data/covid_df.pkl'
df_train, df_test = pipeline.read_and_process_data(filepath)
del filepath

In [13]:
pipeline.sanity_check(df_train, df_test)

Success: Features match


Checking variables with highest percentage of NAs in training set:

In [14]:
df_train.apply(lambda x: x.isnull().sum()/len(x)).sort_values(ascending=False)[:20]

E4_International support                 0.159124
H4_Emergency investment in healthcare    0.149890
H5_Investment in vaccines                0.139698
H2_Testing policy                        0.136277
E2_Debt/contract relief                  0.134989
E3_Fiscal measures                       0.134327
H3_Contact tracing                       0.134106
E1_Income support                        0.131862
C5_Close public transport                0.131199
C7_Restrictions on internal movement     0.131163
C4_Restrictions on gatherings            0.131163
C3_Cancel public events                  0.131163
C8_International travel controls         0.131126
C6_Stay at home requirements             0.131126
H1_Public information campaigns          0.131126
C2_Workplace closing                     0.131126
Diabetes Prevalence                      0.127925
Physicians per 1k                        0.127925
Universal Health Care Coverage Index     0.127925
C1_School closing                        0.127925


In [15]:
#import importlib
#importlib.reload(pipeline)
df_train = df_train.dropna()
df_test = df_test.dropna()
x_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
x_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')

In [26]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_test)

In [37]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)


0.904898841538611